# Capítulo 3: Preparación

Se normalizan nombres de columnas, se verifican/crean claves de georreferencia del **DANE** y se ajusta el formato del código departamental (`DPTO_CCDGO`, 2 dígitos con `zfill(2)`).  
Si no hay columna de código, se intenta inferir a partir del nombre oficial del departamento. También se revisan duplicados y valores faltantes, y se define la **variable objetivo** (por ejemplo, `DIRECTORIO` para conteo o una variable numérica para promedio).

**Objetivo:** dejar el dataframe listo para unir con el shapefile.  
**Entradas:** `data/CHC_2021_raw.parquet`.  
**Salidas:** dataframe preparado con `DPTO_CCDGO` y la variable objetivo definida.


In [9]:
# ===== 3.1 Imports + Carga =====
import re
import pandas as pd

# Carga desde Parquet (necesita pyarrow o fastparquet)
# Si no lo tienes: conda install -c conda-forge pyarrow -y
ruta_parquet = "../data/CHC_2021_raw.parquet"

df = pd.read_parquet(ruta_parquet)
print("Cargado:", df.shape)
df.head()


Cargado: (6250, 130)


,DIRECTORIO,TIP_FOR,P1,P1S1,P2,P2S1,P5,CTL_1,P8R,P9,...,P35,P36R,P37S1,P37S2,P37S3,P37S4,P37S5,P37S6,P37S7,COMPLETA
0,1.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,44.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,33.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,3.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,41.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,4.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,29.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,5.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,30.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## Vista previa del dataset cargado

El dataset tiene un total de **6.250 filas y 130 columnas**.  
A continuación, se muestran las primeras 5 filas:

| DIRECTORIO | TIP_FOR | P1   | P1S1    | P2  | P2S1 | P5  | CTL_1 | P8R | P9  | ... | P35 | P36R | P37S1 | P37S2 | P37S3 |
|------------|---------|------|---------|-----|------|-----|-------|-----|-----|-----|-----|------|-------|-------|-------|
| 1.0        | 2.0     | 85.0 | 85001.0 | 1.0 | 5.0  | 2.0 | 1.0   | 44.0| 1.0 | ... | NaN | NaN  | NaN   | NaN   | NaN   |
| 2.0        | 2.0     | 85.0 | 85001.0 | 1.0 | 5.0  | 2.0 | 1.0   | 33.0| 1.0 | ... | NaN | NaN  | NaN   | NaN   | NaN   |
| 3.0        | 2.0     | 85.0 | 85001.0 | 1.0 | 5.0  | 2.0 | 1.0   | 41.0| 1.0 | ... | NaN | NaN  | NaN   | NaN   | NaN   |
| 4.0        | 2.0     | 85.0 | 85001.0 | 1.0 | 5.0  | 2.0 | 1.0   | 29.0| 1.0 | ... | NaN | NaN  | NaN   | NaN   | NaN   |
| 5.0        | 2.0     | 85.0 | 85001.0 | 1.0 | 5.0  | 2.0 | 1.0   | 30.0| 1.0 | ... | NaN | NaN  | NaN   | NaN   | NaN   |

### Descripción de algunas variables:
- **DIRECTORIO**: identificador único de cada hogar/encuesta.  
- **TIP_FOR**: tipo de formulario aplicado.  
- **P1, P2, P5...**: preguntas de la encuesta con valores numéricos.  
- **P1S1, P2S1...**: sub-preguntas o variables derivadas.  
- **CTL_1**: posible variable de control.  
- **P8R, P9...**: variables de respuestas específicas.  
- **P35, P36R, P37S...**: columnas con presencia de valores faltantes (`NaN`), lo que indica que deberán ser tratadas en el preprocesamiento.

---

### Observaciones
- El dataset es extenso y contiene tanto valores completos como una gran cantidad de **NaN** en columnas finales.  
- Es necesario realizar un **proceso de limpieza** y, probablemente, seleccionar variables clave para el análisis.  
- Este dataset podrá enlazarse con información geográfica (shapefile) para análisis espaciales o regionales.


In [11]:
# ===== 3.X Construir DPTO_CCDGO de forma robusta =====
import re, difflib
import pandas as pd

# --- Diccionario oficial (codigo->nombre) y helpers ---
dane_deptos = {
    '05':'ANTIOQUIA','08':'ATLÁNTICO','11':'BOGOTÁ, D.C.','13':'BOLÍVAR','15':'BOYACÁ',
    '17':'CALDAS','18':'CAQUETÁ','19':'CAUCA','20':'CESAR','23':'CÓRDOBA','25':'CUNDINAMARCA',
    '27':'CHOCÓ','41':'HUILA','44':'LA GUAJIRA','47':'MAGDALENA','50':'META','52':'NARIÑO',
    '54':'NORTE DE SANTANDER','63':'QUINDÍO','66':'RISARALDA','68':'SANTANDER','70':'SUCRE',
    '73':'TOLIMA','76':'VALLE DEL CAUCA','81':'ARAUCA','85':'CASANARE','86':'PUTUMAYO',
    '88':'SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA','91':'AMAZONAS','94':'GUAINÍA',
    '95':'GUAVIARE','97':'VAUPÉS','99':'VICHADA'
}
def _norm(s):
    if pd.isna(s): return None
    s = str(s).strip().upper()
    for a,b in (('Á','A'),('É','E'),('Í','I'),('Ó','O'),('Ú','U')): s = s.replace(a,b)
    s = re.sub(r'\s+', ' ', s)
    return s

nombre_a_codigo = {_norm(v): k for k,v in dane_deptos.items()}

def cod2(s):
    if pd.isna(s): return None
    s = re.sub(r'\D','', str(s).strip())
    return s[:2].zfill(2) if len(s) >= 2 else None

print("Columnas del df:", list(df.columns)[:50], "...")

# 1) Si ya existe DPTO_CCDGO, estandariza y termina
if 'DPTO_CCDGO' in df.columns:
    df['DPTO_CCDGO'] = df['DPTO_CCDGO'].astype(str).str.zfill(2)
else:
    asignada = False

    # 2) Heurística códigos numéricos (COGEMOS 2 PRIMEROS DÍGITOS)
    cobertura_cod = []
    for c in df.columns:
        # intentamos extraer dígitos
        try:
            tmp = df[c].map(lambda x: re.sub(r'\D','', str(x)) if pd.notna(x) else "")
        except Exception:
            continue
        # ¿parece un código municipal (5 dígitos) o al menos 2?
        validos5 = (tmp.str.len()>=5).mean()
        validos2 = (tmp.str.len()>=2).mean()
        if max(validos5, validos2) > 0.6:  # 60% de las filas tienen algo tipo código
            cobertura_cod.append((max(validos5, validos2), c))
    cobertura_cod.sort(reverse=True)

    if cobertura_cod:
        mejor_col = cobertura_cod[0][1]
        print(f"Usando columna numérica candidata: {mejor_col}")
        df['DPTO_CCDGO'] = df[mejor_col].map(cod2)
        asignada = df['DPTO_CCDGO'].notna().any()

    # 3) Si no hubo suerte con códigos, probamos por NOMBRE con fuzzy
    if not asignada:
        # candidatas de texto con pocos valores únicos (evita columnas libres)
        candidatas_texto = [c for c in df.select_dtypes(include='object').columns
                            if df[c].nunique(dropna=True) <= 50]  # ajusta si tu df es muy grande
        print("Candidatas texto:", candidatas_texto[:20])

        mejor = None
        mejor_cob = 0
        for c in candidatas_texto:
            vals = df[c].dropna().astype(str).map(_norm).unique().tolist()
            aciertos = 0
            for v in vals:
                if v in nombre_a_codigo: 
                    aciertos += 1
                else:
                    # fuzzy: buscar el nombre de depto más cercano
                    match = difflib.get_close_matches(v, list(nombre_a_codigo.keys()), n=1, cutoff=0.82)
                    if match: aciertos += 1
            cobertura = aciertos / max(1,len(vals))
            if cobertura > mejor_cob:
                mejor_cob, mejor = cobertura, c

        if mejor and mejor_cob >= 0.5:  # al menos 50% de cobertura
            print(f"Usando columna de nombres: {mejor} (cobertura ~{mejor_cob:.0%})")
            def map_nombre(x):
                vx = _norm(x)
                if vx in nombre_a_codigo: 
                    return nombre_a_codigo[vx]
                match = difflib.get_close_matches(vx, list(nombre_a_codigo.keys()), n=1, cutoff=0.82)
                return nombre_a_codigo[match[0]] if match else None
            df['DPTO_CCDGO'] = df[mejor].map(map_nombre)
            asignada = df['DPTO_CCDGO'].notna().any()

    # 4) Si aún faltan filas sin código, listamos para completar a mano
    if 'DPTO_CCDGO' in df.columns:
        df['DPTO_CCDGO'] = df['DPTO_CCDGO'].astype(str).str.zfill(2)
        faltan = df['DPTO_CCDGO'].isna().sum() if df['DPTO_CCDGO'].dtype != 'object' else (df['DPTO_CCDGO']=="").sum()
    else:
        faltan = len(df)

    if faltan:
        print(f"Faltan asignar código en {faltan} filas. Te muestro valores únicos por la mejor columna de texto (si hay):")
        if not asignada and df.select_dtypes(include='object').shape[1] > 0:
            # muestra algunas categorías que podrías mapear
            col_demo = df.select_dtypes(include='object').columns[0]
            print("Ejemplos en", col_demo, ":", df[col_demo].dropna().astype(str).unique()[:30])

        if 'DPTO_CCDGO' not in df.columns:
            raise ValueError("No se pudo inferir DPTO_CCDGO automáticamente. "
                             "Revisa las columnas y crea un mapeo manual (abajo).")

# 5) Chequeo final
df['DPTO_CCDGO'] = df['DPTO_CCDGO'].astype(str).str.zfill(2)
print("OK DPTO_CCDGO creado. Ejemplo:")
df[['DPTO_CCDGO']].head()


Columnas del df: ['DIRECTORIO', 'TIP_FOR', 'P1', 'P1S1', 'P2', 'P2S1', 'P5', 'CTL_1', 'P8R', 'P9', 'P10R', 'P11', 'P12', 'P13', 'P15', 'P16S1', 'P16S2', 'P16S3', 'P16S4', 'P16S5', 'P16S6', 'P16S7', 'P16S8', 'P16S9', 'P17', 'P17S1', 'P17S2', 'P17S3', 'P17S4', 'P17S5', 'P17S6', 'P17S7', 'P17S8', 'P17S9', 'P17S10', 'P18', 'P19', 'P20', 'P20S1', 'P20S2', 'P20S3', 'P20S4', 'P20S5', 'P20S1A1', 'P20S2A1', 'P20S3A1', 'P20S4A1', 'P20S5A1', 'P21', 'P22'] ...
Usando columna numérica candidata: TIP_FOR
OK DPTO_CCDGO creado. Ejemplo:


,DPTO_CCDGO
0,20
1,20
2,20
3,20
4,20


In [12]:
# Faltantes por columna (tabla para el informe)
tabla_na = df.isna().sum().reset_index()
tabla_na.columns = ['columna','faltantes']
tabla_na.sort_values('faltantes', ascending=False).head(20)

,columna,faltantes
50,P23,6250
37,P20,6248
100,P32S4,6248
105,P33,6246
102,P32S6,6246
113,P33_2,6245
101,P32S5,6241
31,P17S7,6233
28,P17S4,6232
99,P32S3,6226


## Análisis de valores faltantes

Se realizó un conteo de los valores faltantes (`NaN`) en las columnas del dataset.  
En la tabla se listan las variables con mayor cantidad de datos faltantes:

| columna | faltantes |
|---------|-----------|
| P23     | 6250      |
| P20     | 6248      |
| P32S4   | 6248      |
| P33     | 6246      |
| P32S6   | 6246      |
| P33_2   | 6245      |
| P32S5   | 6241      |
| P17S7   | 6233      |
| P17S4   | 6232      |
| P32S3   | 6226      |
| P30     | 6226      |
| P17S1   | 6222      |
| P32S2   | 6217      |
| P37S6   | 6213      |
| P20S3A1 | 6211      |
| P26     | 6207      |
| P17S3   | 6188      |
| P20S5A1 | 6186      |
| P37S7   | 6167      |
| P20S4A1 | 6164      |

### Observaciones
- Algunas columnas como **P23** tienen **el 100% de valores faltantes (6250 de 6250)**, por lo que no aportan información útil.  
- La mayoría de estas variables corresponden a **preguntas específicas o sub-preguntas** que probablemente no aplican para todos los hogares/encuestas.  
- Existen grupos de preguntas relacionadas (ej. `P32S*`, `P17S*`, `P20S*`) que muestran un patrón de ausencia generalizada.  
- Es recomendable:
  - **Eliminar columnas** con más del 95–100% de datos faltantes.  
  - Evaluar si las variables con faltantes parciales (ej. 6164 a 6248) pueden ser imputadas o si conviene descartarlas según el análisis.

---

### Conclusión preliminar
El dataset contiene un conjunto considerable de variables **con escasa información registrada**. Una depuración inicial eliminaría varias columnas, lo cual simplificaría el análisis posterior y reduciría ruido en los modelos estadísticos o visualizaciones.


In [13]:
# Duplicados por DPTO_CCDGO (si esperas un valor único por departamento)
dup_por_codigo = df.groupby('DPTO_CCDGO').size().reset_index(name='conteo')
dup_por_codigo[dup_por_codigo['conteo']>1].sort_values('conteo', ascending=False).head(10)

,DPTO_CCDGO,conteo
0,20,6248
1,None,2


## Revisión de la variable `DPTO_CCDGO`

Se generó un conteo de frecuencias para la columna **`DPTO_CCDGO`**:

| DPTO_CCDGO | conteo |
|------------|--------|
| 20         | 6248   |
| None       | 2      |

### Observaciones
- El **código de departamento 20** concentra la gran mayoría de registros (**6248 de 6250**), lo que indica que casi todo el dataset pertenece a un mismo departamento.  
- Existen **2 registros sin información (`None`)**, que deben ser tratados:
  - Se pueden **eliminar** si representan un porcentaje insignificante (0.03%).  
  - Alternativamente, se podrían **asignar manualmente** si se conoce a qué departamento corresponden.  

### Conclusión
La variable `DPTO_CCDGO` no está distribuida en múltiples departamentos como se esperaba, sino que concentra casi todos los datos en el código **20**. Esto limita la posibilidad de análisis comparativo entre regiones y sugiere que el dataset solo cubre un departamento específico o que la codificación no se hizo correctamente.
